In [2]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [10]:
import csv
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose

action_label = "sentarse"

video_path = 'sentarseFrente.mp4'
cap = cv2.VideoCapture(video_path)

video_name = video_path.split('/')[-1]

frame_count = 0

with open('sentarseFrente.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    header = ['label', 'frame_number', 'timestamp', 'video_name']
    for i in range(33):
        header += [f'x{i}', f'y{i}', f'z{i}']
    writer.writerow(header)

    with mp_pose.Pose() as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = pose.process(rgb_frame)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
                row = [action_label, frame_count, timestamp, video_name]

                for landmark in landmarks:
                    row.append(landmark.x)
                    row.append(landmark.y)
                    row.append(landmark.z)

                writer.writerow(row)

            frame_count += 1

cap.release()
cv2.destroyAllWindows()


In [11]:
import os
import pandas as pd

# Ruta de la carpeta donde tienes tus archivos CSV
carpeta_csv = './'

# Lista para almacenar todos los DataFrames
dataframes = []

# Iterar sobre todos los archivos en la carpeta
for archivo in os.listdir(carpeta_csv):
    if archivo.endswith('.csv'):
        # Leer cada archivo CSV y agregarlo a la lista de DataFrames
        df = pd.read_csv(os.path.join(carpeta_csv, archivo))
        dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame resultante en un nuevo archivo CSV
df_concatenado.to_csv('todos_los_datos.csv', index=False)

print('Todos los archivos CSV han sido combinados en "todos_los_datos.csv"')


Todos los archivos CSV han sido combinados en "todos_los_datos.csv"


#EDA

In [14]:
import pandas as pd

ruta_csv = 'finalDataset.csv'

df = pd.read_csv(ruta_csv)

df.describe()

,frame_number,timestamp,x0,y0,z0,x1,y1,z1,x2,y2,...,z29,x30,y30,z30,x31,y31,z31,x32,y32,z32
count,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,...,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000
mean,84.311346,2.810195,0.425957,0.324484,-0.145194,0.430180,0.314298,-0.147316,0.433253,0.314225,...,0.140277,0.462297,0.797022,0.207657,0.477035,0.812807,0.006833,0.423069,0.816419,0.077756
std,64.913796,2.163887,0.086387,0.068552,0.286800,0.086630,0.067988,0.257104,0.086769,0.067704,...,0.307084,0.064152,0.036115,0.300315,0.124997,0.043716,0.298621,0.101871,0.046296,0.306394
min,0.000000,0.000000,0.049825,0.173078,-0.669725,0.046200,0.164278,-0.625409,0.049854,0.165022,...,-0.975791,0.328137,0.705603,-1.048413,0.168815,0.715437,-1.091620,0.185957,0.709667,-1.132987
25%,35.000000,1.166667,0.403012,0.293687,-0.389617,0.408445,0.285261,-0.359589,0.410615,0.285364,...,-0.076428,0.423888,0.773522,-0.002987,0.410197,0.781733,-0.160880,0.385805,0.790830,-0.148879
50%,71.000000,2.366667,0.439119,0.307824,-0.145766,0.448022,0.298273,-0.153409,0.451370,0.298142,...,0.132903,0.462729,0.792447,0.264837,0.511695,0.797496,0.016555,0.428475,0.803134,0.158704
75%,110.000000,3.666667,0.475231,0.384151,-0.009205,0.479925,0.371360,-0.023702,0.481355,0.371462,...,0.399909,0.501961,0.817149,0.412244,0.571792,0.846270,0.246111,0.488002,0.850444,0.312670
max,279.000000,9.300000,0.595567,0.496974,1.098962,0.593350,0.491552,1.110413,0.588360,0.489401,...,0.855844,0.634061,0.872170,0.768167,0.635142,0.882247,0.811081,0.638353,0.893808,0.608747
